# Setup

In [3]:
from parameters.parameters import prune_quantiles, prune_measure

debug = True # If True, will produce plots and/or verbose output to double-check

import sys
sys.path.append("..")

# Path
print("Current working directory: ", sys.path[0])
%run -i "../parameters/parameters.py"
%run -i "../src/path.py"
%run -i "../src/setup.py"
%run -i "../src/functions.py"
%run -i "greedy_triangulation.py"
%run -i "graph_similarity.py"

%load_ext watermark
%watermark -n -v -m -g -iv

Current working directory:  /Users/jaborup/Projects/continuous-bike-growth
Loaded parameters.

Loaded PATH.



=== Cities ===
{   'montreal': {   'countryid': 'can',
                    'name': 'Montreal',
                    'nominatimstring': 'montreal (06)'}}


Setup finished.

Loaded functions.

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
osgeo.gdal 3.8.5
csv        1.0
igraph     0.11.6
networkx   3.3
fiona      1.9.5
shapely    2.0.4
geopandas  1.0.1
pyproj     3.6.1
watermark  2.0.2
osmnx      2.0.0b3
numpy      1.26.4
matplotlib 3.8.4
pandas     2.2.3
geojson    2.5.0
Thu Nov 21 2024 

CPython 3.12.7
IPython 8.6.0

compiler   : Clang 17.0.6 
system     : Darwin
release    : 23.6.0
machine    : arm64
processor  : arm
CPU cores  : 8
interpreter: 64bit
Git hash   : c612c28e8968e42c6cb99a6dda6fb984fecb43e7


# Run the program

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(placeid + ": Generating graph and POIs")

    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]

    numIterations = 2
    subgraph_percentages = [0.6, 0.4]
    prune_quantiles = [0.5]
    for prune_measure in ["betweenness", "random", "closeness"]:
        print(f"Prune measure: {prune_measure}")

        (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

        print(f"Writing data to {placeid}_{prune_measure}.csv")
        with open(f'{placeid}_{prune_measure}.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow([
                "sum_of_errors",
                "continuous_vertices",
                "bikengrowth_vertices",
                "continuous_disconnected_pairs",
                "bikengrowth_disconnected_pairs",
            ])

            for i in range(100):
                print(f"Running iteration {i + 1}")
                if prune_measure == "random":
                    (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

                gts, abstract_gts = greedy_triangulation_in_steps(G_carall, nnids, numIterations, subgraph_percentages, prune_quantiles, prune_measure)

                for prune_quantile, gt, szell_gt in zip(prune_quantiles, gts, GTs):
                    error, our_disconnected_points, szell_disconnected_points = sum_of_errors_pois(szell_gt, gt, nnids)

                writer.writerow([error, gt.vcount(), szell_gt.vcount(), our_disconnected_points, szell_disconnected_points])

Cities:   0%|          | 0/1 [00:00<?, ?it/s]

montreal: Generating graph and POIs


/Users/jaborup/Projects/continuous-bike-growth/src/functions.py:345: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  n = pd.read_csv(p + prefix + '_nodes.csv')
/Users/jaborup/Projects/continuous-bike-growth/src/functions.py:346: DtypeWarning: Columns (5,9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  e = pd.read_csv(p + prefix + '_edges.csv')


Prune measure: betweenness


Greedy triangulation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Vertices after copying original graph: 143
Vertices after first greedy triangulation: 26229
Vertices after first pruning: 17811
Vertices after second greedy triangulation: 33631
Vertices after second pruning: 17811
Vertices in shortest paths: 17811


Running iteration 38


Greedy triangulation on 60.0% subgraph:   0%|          | 0/1 [00:00<?, ?it/s]

Vertices after copying original graph: 143
Vertices after first greedy triangulation: 26261
Vertices after first pruning: 16644
Vertices after second greedy triangulation: 34010
Vertices after second pruning: 16644
Vertices in shortest paths: 16644


Running iteration 39


In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    result_file = open(PATH["results"] + placeid + "/" + filename + ".pickle", 'rb')
    res = pickle.load(result_file)
    szell_gts = res["GTs"]